# 並行模型鏈 (Parallel Model Chains)

本範例展示如何使用 RunnableParallel 同時調用多個模型進行不同角度的分析。

## 🎯 學習目標
- 理解並行處理的概念和優勢
- 掌握 RunnableParallel 的使用方法
- 學習如何合併多個模型的分析結果

## 📝 核心概念
並行模型鏈使用 RunnableParallel 同時執行多個模型調用，每個模型負責不同的分析任務，最後合併結果，大幅提升處理效率。

## ✨ 適用場景
- 多角度內容分析（品質、語氣、情感等）
- 需要同時獲取多種評估結果
- 提升處理效率的場景
- 獨立任務的並行處理

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")
print("✅ 模型已建立")

In [ ]:
# 主提示模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

# 格式化函數
def format_email(reply):
    return f"""
親愛的客戶，

{reply}

感謝您的來信。

此致
客服團隊
"""

# 品質檢查鏈
quality_check_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質審查專家。請評估這封郵件的專業度，給出1-10分評分和簡短評語。"),
    ("human", "評估這封郵件：\n\n{email}")
])
quality_check_chain = quality_check_prompt | model | StrOutputParser()

# 語氣分析鏈
tone_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是語氣分析專家。請分析這封郵件的語氣是否友善、專業。"),
    ("human", "分析這封郵件的語氣：\n\n{email}")
])
tone_analysis_chain = tone_analysis_prompt | model | StrOutputParser()

print("✅ 分析鏈已建立")

In [ ]:
# 使用 RunnableParallel 並行執行
parallel_chain = RunnableParallel(
    formatted_email=RunnableLambda(format_email),
    quality_check=RunnableLambda(lambda x: {"email": format_email(x)}) | quality_check_chain,
    tone_analysis=RunnableLambda(lambda x: {"email": format_email(x)}) | tone_analysis_chain
)

# 合併結果
def combine_results(parallel_results):
    return f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{parallel_results['formatted_email']}

{'='*60}
📊 AI 品質檢查結果：
{'='*60}
{parallel_results['quality_check']}

{'='*60}
🎭 AI 語氣分析結果：
{'='*60}
{parallel_results['tone_analysis']}
"""

# 完整鏈
parallel_model_chain = (
    prompt_template 
    | model 
    | StrOutputParser() 
    | parallel_chain 
    | RunnableLambda(combine_results)
)

print("✅ 並行模型鏈已建立！")
print("流程：主 Prompt → 主模型 → 並行處理（格式化 + 品質檢查 + 語氣分析）→ 合併結果")

In [ ]:
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

print("="*60)
print("並行模型鏈測試結果：")
print("="*60)

result = parallel_model_chain.invoke({"email_content": customer_email})
print(result)

## 📊 並行模型鏈總結

### ✅ 優點
- **效率高**：並行執行，節省總體時間
- **多角度分析**：同時從不同角度評估內容
- **結構清晰**：每個分析任務獨立且明確

### ⚠️ 注意事項
- **資源消耗**：同時調用多個模型會增加資源使用
- **需要合併邏輯**：需要額外處理來合併結果

### 🔗 相關技術
- [串聯模型鏈](5_chains_sequential_model_ollama.ipynb)：順序執行
- [閉包模型鏈](7_chains_closure_model_ollama.ipynb)：靈活控制
- [動態提示鏈](9_chains_dynamic_prompt_ollama.ipynb)：智能路由